In [3]:
from datasets import load_dataset, Dataset
import pandas as pd 
import torch
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction

In [2]:
# import relativedelta

from dateutil.relativedelta import relativedelta

In [5]:
df = pd.read_csv("/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_2_transformer.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['Hour of the day'] = df['Date'].dt.hour
df['Day of the week'] = df['Date'].dt.dayofweek

In [3]:
df = pd.read_csv("/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/EPEX_FR 2.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['Hour of the day'] = df['Date'].dt.hour
df['Day of the week'] = df['Date'].dt.dayofweek
print(df.head())

window = 1 # 1 year
context_length = 24*30 # 1 week
pred_date = pd.to_datetime('2021-01-01')
df['Date'].iloc[0] + pd.Timedelta(hours=context_length)
date_range = pd.date_range(start=pred_date-relativedelta(years=1)+ pd.Timedelta(hours=context_length), end=pred_date-pd.Timedelta(hours=1), freq='d')

Dataset_ = {'past_values':[], 'future_values':[], 'past_time_features':[], 'future_time_features':[], 'past_observed_mask':[]}

for date in date_range:
    mask_date_context = (df['Date'] >= date-relativedelta(hours=context_length)) & (df['Date'] < date)
    mask_date_pred = (df['Date'] >= date) & (df['Date'] < date+relativedelta(hours=24))
    past_values = torch.tensor(df['Price'][mask_date_context].values)
    future_values = torch.tensor(df['Price'].loc[mask_date_pred].values)

    past_time_features = torch.tensor(df[['Hour of the day', 'Day of the week', 'Load forecast', 'Generation forecast']][mask_date_context].values)
    future_time_features = torch.tensor(df[['Hour of the day', 'Day of the week', 'Load forecast', 'Generation forecast']][mask_date_pred].values)

    past_observed_mask = torch.ones(context_length)

    Dataset_['past_values'].append(past_values)
    Dataset_['future_values'].append(future_values)
    Dataset_['past_time_features'].append(past_time_features)
    Dataset_['future_time_features'].append(future_time_features)
    Dataset_['past_observed_mask'].append(past_observed_mask)

Dataset_['past_values'] = torch.stack(Dataset_['past_values']).to(dtype=torch.float32)
Dataset_['future_values'] = torch.stack(Dataset_['future_values']).to(dtype=torch.float32)
Dataset_['past_time_features'] = torch.stack(Dataset_['past_time_features']).to(dtype=torch.float32)
Dataset_['future_time_features'] = torch.stack(Dataset_['future_time_features']).to(dtype=torch.float32)
Dataset_['past_observed_mask'] = torch.stack(Dataset_['past_observed_mask']).to(dtype=torch.float32)

                 Date  Load forecast  Generation forecast  Price   
0 2017-01-01 00:00:00        73650.0              69539.0  58.23  \
1 2017-01-01 01:00:00        72350.0              67376.0  51.95   
2 2017-01-01 02:00:00        68750.0              65412.0  47.27   
3 2017-01-01 03:00:00        65900.0              64557.0  45.49   
4 2017-01-01 04:00:00        65000.0              64552.0  44.50   

   Hour of the day  Day of the week  
0                0                6  
1                1                6  
2                2                6  
3                3                6  
4                4                6  


In [4]:
print(Dataset_['past_time_features'].shape)
print(Dataset_['past_values'].shape)
print(Dataset_['past_observed_mask'].shape)
print(Dataset_['future_time_features'].shape)
print(Dataset_['future_values'].shape)



torch.Size([336, 720, 4])
torch.Size([336, 720])
torch.Size([336, 720])
torch.Size([336, 24, 4])
torch.Size([336, 24])


In [5]:
print(past_time_features.shape)
print(past_values.shape)
print(past_observed_mask.shape)

print(future_time_features.shape)
print(future_values.shape)

torch.Size([720, 4])
torch.Size([720])
torch.Size([720])
torch.Size([24, 4])
torch.Size([24])


In [6]:
Dataset_['past_values'][:6]

tensor([[38.6000, 36.5500, 32.3200,  ..., 30.4300, 29.2800, 23.6100],
        [31.9800, 30.5000, 28.7900,  ..., 28.7400, 24.1200, 14.7100],
        [30.7500, 29.0300, 24.9600,  ..., 17.2400, 28.5600, 23.1400],
        [32.0500, 30.1400, 28.5500,  ..., 23.8700, 17.3800, 15.9200],
        [32.2800, 31.1800, 30.1000,  ..., 27.1300, 24.7600, 20.7900],
        [29.0000, 29.0800, 27.7200,  ..., 30.3600, 32.4000, 25.4400]])

In [ ]:
past_values

In [14]:


# Initializing a Time Series Transformer configuration with 24 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=24, context_length=24*29, input_size=1, output_size=1, num_time_features=4)

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerForPrediction(configuration)

# Accessing the model configuration
configuration = model.config

In [15]:


# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=Dataset_['past_values'],
    past_time_features=Dataset_['past_time_features'],
    past_observed_mask=Dataset_['past_observed_mask'],
    future_values=Dataset_['future_values'],
    future_time_features=Dataset_['future_time_features']
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=Dataset_['past_values'],
    past_time_features=Dataset_['future_time_features'],
    past_observed_mask=Dataset_['past_observed_mask'],
    future_time_features=Dataset_['future_time_features']
)

mean_prediction = outputs.sequences.mean(dim=1)

ValueError: input length 720 and time feature lengths 737 does not match

In [ ]:
mean_prediction

tensor([[ -7.5163, -63.3271,   2.6899, -16.8701,  -2.7818,  -3.5425,  14.8743,
           9.1403,  -4.4051,  12.6202,   7.6990, -19.4491, -16.7331,  15.3230,
          -9.7587,  -0.2657,  17.9799, -12.4463,   5.1261,  11.6024,   9.9855,
         -22.2234,   5.5988,  13.4501],
        [ -9.2073,  -8.1139,  12.3300,   6.4870,  17.6020,  -4.9579, -17.7479,
          22.4583,  10.6285,   1.9875,  13.6067,  -0.3630,   6.9221,  24.8311,
          -2.5540,  11.2675,  -2.2600,  -7.6559,  14.9481,   1.0363,   0.4891,
           4.6454, -20.1811,  17.2088],
        [-42.6548,  20.2999,  28.9912,  15.0008,  17.3160, -33.3663,   8.9611,
           8.9200, -15.7974,   0.3519,   3.6403,  19.3589,  45.0078,   4.0417,
          -9.7578,  14.4422,   3.3531,  26.9713,  -0.9255, -13.8006, -34.6340,
         -18.0744, -14.3971, -15.8072],
        [  4.5246,  -9.2025,  -5.2003,  19.7376,   6.2186,  15.8372,  -6.1950,
           2.9575,   2.7287, -14.9392,  37.3583, -11.7959,   0.2238,  28.6783,
           

In [ ]:
model.num_parameters()

214339

In [43]:
dataset = Dataset.from_pandas(df)

dataset = dataset.map(lambda x: {'Day of the week': x['Date'].dayofweek})
dataset = dataset.map(lambda x: {'Hour of the day': x['Date'].hour})


In [44]:
train_dataset = dataset.filter(lambda x: x['Date'].year < 2019)
test_dataset = dataset.filter(lambda x: x['Date'].date() == pd.to_datetime('2019-01-01').date())

In [18]:
train_dataset

Dataset({
    features: ['Date', 'Load forecast', 'Generation forecast', 'Price', 'Hour of the day', 'Day of the week'],
    num_rows: 17520
})

In [19]:
date = train_dataset['Date'][24*30].date()

past_values_all = train_dataset.filter(lambda x: x['Date'].date() < date)
past_observed_mask = torch.ones(24*30)
#past_observed_mask[-24:,-1] = 0
past_time_features = torch.tensor(list(zip(past_values_all['Day of the week'], past_values_all['Hour of the day'], past_values_all['Load forecast'], past_values_all['Generation forecast'])))
past_values = torch.tensor(past_values_all['Price'])



future_values_all = train_dataset.filter(lambda x: x['Date'].date() == date)
future_values = torch.tensor(future_values_all['Price'])

future_time_features = torch.tensor(list(zip(future_values_all['Day of the week'], future_values_all['Hour of the day'], future_values_all['Load forecast'], future_values_all['Generation forecast'])))



In [20]:
train_dataset['Date'][24*30].date()

datetime.date(2017, 1, 31)

In [21]:
number_of_days =1
for i in range(number_of_days):
    date = date + pd.Timedelta(hours=24)
    past_values_all_2 = train_dataset.filter(lambda x: date - pd.Timedelta(days = 30) <= x['Date'].date() < date)
    past_observed_mask_temp = torch.ones(24*30)
    past_observed_mask=torch.cat((past_observed_mask, past_observed_mask_temp), dim=0)#.reshape(i+3, 24*16, 3)
    past_time_features = torch.cat((past_time_features, torch.tensor(list(zip(past_values_all_2['Day of the week'], past_values_all_2['Hour of the day'], past_values_all_2['Load forecast'], past_values_all_2['Generation forecast'])))))#.reshape(i+3, 24*16, 2)
    past_values = torch.cat((past_values, torch.tensor(past_values_all_2['Price'])))#.reshape(i+3, 24*16, 3)
    
    future_values_all_2 = train_dataset.filter(lambda x: x['Date'].date() == date)
    future_values = torch.cat((future_values, torch.tensor(future_values_all_2['Price'])))#.reshape(i+3, 24, 3)
    future_time_features = torch.cat((future_time_features, torch.tensor(list(zip(future_values_all_2['Day of the week'], future_values_all_2['Hour of the day'], future_values_all_2['Load forecast'], future_values_all_2['Generation forecast'])))))#.reshape(i+3, 24, 2)

past_values = past_values.reshape(number_of_days+1, 24*30)
past_time_features = past_time_features.reshape(number_of_days+1, 24*30, 4)
past_observed_mask = past_observed_mask.reshape(number_of_days+1, 24*30)

future_values = future_values.reshape(number_of_days+1, 24)
future_time_features = future_time_features.reshape(number_of_days+1, 24, 4)



In [22]:
print(past_time_features.shape)
print(past_values.shape)
print(past_observed_mask.shape)

print(future_time_features.shape)
print(future_values.shape)

torch.Size([2, 720, 4])
torch.Size([2, 720])
torch.Size([2, 720])
torch.Size([2, 24, 4])
torch.Size([2, 24])


In [23]:
future_values

tensor([[ 55.0000,  53.3000,  52.0000,  50.1800,  54.0000,  70.2000,  86.6600,
         101.0000,  90.3400,  89.8000,  89.8200,  82.3600,  82.0200,  90.3600,
          89.7500,  90.2000,  95.0000, 104.3300,  90.0000,  83.6900,  60.7700,
          59.6200,  59.2000,  54.7100],
        [ 54.9900,  51.8100,  48.1600,  40.9100,  50.6800,  61.1000,  72.9300,
          78.6400,  79.1100,  79.1100,  78.4400,  76.6400,  75.4700,  72.6100,
          69.0600,  64.2100,  70.3100,  76.1000,  77.2000,  70.8100,  59.6000,
          62.8500,  55.1000,  41.1100]])

In [33]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction

# Initializing a Time Series Transformer configuration with 24 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=24, context_length=24*29+17, input_size=1, output_size=1, num_time_features=4)

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerForPrediction(configuration)

# Accessing the model configuration
configuration = model.config

In [44]:
device = torch.device('cuda:2') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

TimeSeriesTransformerForPrediction(
  (model): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=13, out_features=64, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(737, 64)
      (layers): ModuleList(
        (0-1): 2 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=64, out_features=32,

In [8]:
TimeSeriesTransformerConfig?


Init signature:
TimeSeriesTransformerConfig(
    prediction_length: Union[int, NoneType] = None,
    context_length: Union[int, NoneType] = None,
    distribution_output: str = 'student_t',
    loss: str = 'nll',
    input_size: int = 1,
    lags_sequence: List[int] = [1, 2, 3, 4, 5, 6, 7],
    scaling: Union[bool, str, NoneType] = 'mean',
    num_dynamic_real_features: int = 0,
    num_static_categorical_features: int = 0,
    num_static_real_features: int = 0,
    num_time_features: int = 0,
    cardinality: Union[List[int], NoneType] = None,
    embedding_dimension: Union[List[int], NoneType] = None,
    encoder_ffn_dim: int = 32,
    decoder_ffn_dim: int = 32,
    encoder_attention_heads: int = 2,
    decoder_attention_heads: int = 2,
    encoder_layers: int = 2,
    decoder_layers: int = 2,
    is_encoder_decoder: bool = True,
    activation_function: str = 'gelu',
    d_model: int = 64,
    dropout: float = 0.1,
    encoder_layerdrop: float = 0.1,
    decoder_layerdrop: float = 0

In [35]:


# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_values=future_values,
    future_time_features=future_time_features
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_time_features=future_time_features
)

mean_prediction = outputs.sequences.mean(dim=1).detach().cpu()
df_pred = pd.DataFrame(mean_prediction.numpy())

In [43]:
df_pred.to_csv('/home/infres/dfitton-21/pfr/fil_rouge/forecasts/transformer_pred.csv')

In [5]:
mean_prediction.shape

NameError: name 'mean_prediction' is not defined

In [37]:
future_values

tensor([[55.0000, 53.3000, 52.0000,  ..., 59.6200, 59.2000, 54.7100],
        [54.9900, 51.8100, 48.1600,  ..., 62.8500, 55.1000, 41.1100],
        [38.7400, 36.9400, 32.3000,  ..., 40.8000, 42.1200, 39.8700],
        ...,
        [29.5000, 28.9400, 29.9000,  ..., 36.6800, 33.2600, 33.5400],
        [32.8500, 30.7200, 31.0300,  ..., 36.7200, 31.1600, 30.4100],
        [29.5000, 29.1700, 29.5900,  ..., 38.5100, 38.2400, 29.4300]])

In [42]:
model.num_parameters()

214339

In [139]:
from huggingface_hub import hf_hub_download
file = hf_hub_download(
    repo_id="kashif/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

In [140]:
print(batch['past_time_features'].shape)
print(batch['past_values'].shape)
print(batch['past_observed_mask'].shape)

print(batch['future_time_features'].shape)
print(batch['future_values'].shape)

torch.Size([64, 61, 2])
torch.Size([64, 61])
torch.Size([64, 61])
torch.Size([64, 24, 2])
torch.Size([64, 24])


In [94]:
batch['future_time_features'][0]

tensor([[ 0.2273,  2.3541],
        [ 0.3182,  2.3560],
        [ 0.4091,  2.3579],
        [ 0.5000,  2.3598],
        [-0.5000,  2.3617],
        [-0.4091,  2.3636],
        [-0.3182,  2.3655],
        [-0.2273,  2.3674],
        [-0.1364,  2.3692],
        [-0.0455,  2.3711],
        [ 0.0455,  2.3729],
        [ 0.1364,  2.3747],
        [ 0.2273,  2.3766],
        [ 0.3182,  2.3784],
        [ 0.4091,  2.3802],
        [ 0.5000,  2.3820],
        [-0.5000,  2.3838],
        [-0.4091,  2.3856],
        [-0.3182,  2.3874],
        [-0.2273,  2.3892],
        [-0.1364,  2.3909],
        [-0.0455,  2.3927],
        [ 0.0455,  2.3945],
        [ 0.1364,  2.3962]])

In [148]:
from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerForPrediction

file = hf_hub_download(
    repo_id="kashif/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_time_features=batch["future_time_features"],
)

mean_prediction = outputs.sequences.mean(dim=1)

In [150]:
mean_prediction.shape

torch.Size([64, 24])

In [152]:
batch['future_values'].shape

torch.Size([64, 24])

In [156]:
torch.mean(torch.abs(mean_prediction - batch['future_values'])/batch['future_values'])

tensor(0.2474)

In [155]:
batch['future_values'].mean()

tensor(13825.9795)

In [9]:

import torch
import pandas as pd
from datasets import Dataset
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction
import numpy as np
print('import done')

import done


In [39]:
# Test py


#device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)
context =720

df = pd.read_csv("/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/EPEX_FR 2.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['Hour of the day'] = df['Date'].dt.hour
df['Day of the week'] = df['Date'].dt.dayofweek
df['Day of the year'] = df['Date'].dt.dayofyear
df['Year'] = df['Date'].dt.year
df['Month of the year'] = df['Date'].dt.month
print(df.head())

df.to_csv("/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/EPEX_FR_2_transformer.csv")
print('csv done')
nb_time_feat = 7

train_df = df[df['Year'] < 2021]
test_df = df[df['Date'].dt.date == pd.to_datetime('2021-01-01').date()]

date = train_df['Date'].iloc[24*context].date()

past_values_all = train_df[train_df['Date'].dt.date < date]
past_observed_mask = np.ones(24*context).reshape(1, -1)
past_time_features = np.array(past_values_all[['Day of the week', 'Hour of the day', 'Day of the year', 'Year', 'Month of the year', 'Load forecast', 'Generation forecast']]).reshape(1, -1, nb_time_feat)#.astype(np.float64)
past_values = np.array(past_values_all['Price']).reshape(1, -1)#.astype(np.float64)

future_values_all = train_df[train_df['Date'].dt.date == date]
future_values = np.array(future_values_all['Price']).reshape(1, -1)#.astype(np.float64)
future_time_features = np.array(future_values_all[['Day of the week', 'Hour of the day', 'Day of the year', 'Year', 'Month of the year', 'Load forecast', 'Generation forecast']]).reshape(1, -1, nb_time_feat)#.astype(np.float64)

past_values = torch.tensor(past_values).double().to(device)
past_time_features = torch.tensor(past_time_features).double().to(device)
past_observed_mask = torch.tensor(past_observed_mask).double().to(device)
future_values = torch.tensor(future_values).double().to(device)
future_time_features = torch.tensor(future_time_features).double().to(device)

#number_of_days =0
#for i in range(number_of_days):
#date = date + pd.Timedelta(hours=24)
#past_values_all_2 = train_dataset.filter(lambda x: date - pd.Timedelta(days = context) <= x['Date'].date() < date)
#past_observed_mask_temp = torch.ones(24*context)
#past_observed_mask=torch.cat((past_observed_mask, past_observed_mask_temp), dim=0)#.reshape(i+3, 24*context, nb_time_feat)
#past_time_features = torch.cat((past_time_features, torch.tensor(list(zip(past_values_all_2['Day of the week'], past_values_all_2['Hour of the day'],past_values_all_2['Day of the year'], past_values_all_2['Year'], past_values_all_2['Month of the year'], past_values_all_2['Load forecast'], past_values_all_2['Generation forecast'])))))#.reshape(i+3, 24*16, 2)
#past_values = torch.cat((past_values, torch.tensor(past_values_all_2['Price'])))#.reshape(i+3, 24*context, 3)

#future_values_all_2 = train_dataset.filter(lambda x: x['Date'].date() == date)
#future_values = torch.cat((future_values, torch.tensor(future_values_all_2['Price'])))#.reshape(i+3, 24, nb_time_feat)
#future_time_features = torch.cat((future_time_features, torch.tensor(list(zip(future_values_all_2['Day of the week'], future_values_all_2['Hour of the day'],future_values_all_2['Day of the year'], future_values_all_2['Year'], future_values_all_2['Month of the year'], future_values_all_2['Load forecast'], future_values_all_2['Generation forecast'])))))#.reshape(i+3, 24, 2)

#past_values = past_values.reshape(number_of_days+1, 24*context)
#past_time_features = past_time_features.reshape(number_of_days+1, 24*context, nb_time_feat)
#past_observed_mask = past_observed_mask.reshape(number_of_days+1, 24*context)

#future_values = future_values.reshape(number_of_days+1, 24)
#future_time_features = future_time_features.reshape(number_of_days+1, 24, nb_time_feat)
print(past_time_features.shape, past_time_features.dtype)
print(past_values.shape, past_values.dtype)
print(past_observed_mask.shape, past_observed_mask.dtype)

print(future_time_features.shape, future_time_features.dtype)
print(future_values.shape, future_values.dtype)

#past_time_features = past_time_features.to(device)
#past_observed_mask = past_observed_mask.to(device)
#future_values = future_values.to(device)
#future_time_features = future_time_features.to(device)






cpu
                 Date  Load forecast  Generation forecast  Price   
0 2017-01-01 00:00:00        73650.0              69539.0  58.23  \
1 2017-01-01 01:00:00        72350.0              67376.0  51.95   
2 2017-01-01 02:00:00        68750.0              65412.0  47.27   
3 2017-01-01 03:00:00        65900.0              64557.0  45.49   
4 2017-01-01 04:00:00        65000.0              64552.0  44.50   

   Hour of the day  Day of the week  Day of the year  Year  Month of the year  
0                0                6                1  2017                  1  
1                1                6                1  2017                  1  
2                2                6                1  2017                  1  
3                3                6                1  2017                  1  
4                4                6                1  2017                  1  
csv done
torch.Size([1, 17280, 7]) torch.float64
torch.Size([1, 17280]) torch.float64
torch.Size([1, 17280]

In [40]:
torch.tensor(np.array(past_values))

tensor([[58.2300, 51.9500, 47.2700,  ..., 46.2200, 41.2700, 38.0200]],
       dtype=torch.float64)

In [41]:
# Initializing a Time Series Transformer configuration with 24 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=24, context_length=24*(context-1)+17, input_size=1, output_size=1, num_time_features=nb_time_feat )

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerForPrediction(configuration)

# Accessing the model configuration
configuration = model.config

model = model.to(device)



# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_values=future_values,
    future_time_features=future_time_features
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_time_features=future_time_features
)

mean_prediction = outputs.sequences.mean(dim=1).detach().cpu()
df_pred = pd.DataFrame(mean_prediction.numpy())

df_pred.to_csv('/home/infres/dfitton-21/pfr/fil_rouge/forecasts/transformer_pred.csv')

RuntimeError: expected scalar type Double but found Float

In [46]:
from itertools import islice
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
from sklearn.manifold import TSNE
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from pytorch_lightning.loggers import CSVLogger
from datasets import load_dataset

from estimator import TransformerEstimator
dataset = get_dataset("electricity")

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction

/Users/dorianfitton/opt/anaconda3/envs/bgd/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
df = pd.read_csv("/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_2_transformer.csv")

print("Starting making prediction")



device = torch.device("cpu")
context =180

#df = pd.read_csv("/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/EPEX_FR 2.csv")
df['Date'] = pd.to_datetime(df['Date'])
#df['Hour of the day'] = df['Date'].dt.hour
#df['Day of the week'] = df['Date'].dt.dayofweek
#df['Day of the year'] = df['Date'].dt.dayofyear
#df['Year'] = df['Date'].dt.year
#df['Month of the year'] = df['Date'].dt.month
print(df.head())

nb_time_feat = 7

dataset = Dataset.from_pandas(df)

#dataset = dataset.map(lambda x: {'Day of the week': x['Date'].dayofweek})
#dataset = dataset.map(lambda x: {'Hour of the day': x['Date'].hour})
#dataset = dataset.map(lambda x: {'Day of the year': x['Date'].dayofyear})
#dataset = dataset.map(lambda x: {'Year': x['Date'].year})
#dataset = dataset.map(lambda x: {'Month of the year': x['Date'].month})


train_dataset = dataset.filter(lambda x: x['Date'].year < 2021)
test_dataset = dataset.filter(lambda x: x['Date'].date() == pd.to_datetime('2021-01-01').date())

date = train_dataset['Date'][24*context].date()

past_values_all = train_dataset.filter(lambda x: x['Date'].date() < date)
past_observed_mask = torch.ones(24*context).reshape(1, -1)
#past_observed_mask[-24:,-1] = 0
past_time_features = torch.tensor(list(zip(past_values_all['Day of the week'], past_values_all['Hour of the day'],past_values_all['Day of the year'], past_values_all['Year'], past_values_all['Month of the year'] ,past_values_all['Load forecast'], past_values_all['Generation forecast']))).reshape(1, -1, nb_time_feat)
past_values = torch.tensor(past_values_all['Price']).reshape(1, -1) 



future_values_all = train_dataset.filter(lambda x: x['Date'].date() == date)
future_values = torch.tensor(future_values_all['Price']).reshape(1, -1)

future_time_features = torch.tensor(list(zip(future_values_all['Day of the week'], future_values_all['Hour of the day'], future_values_all['Day of the year'], future_values_all['Year'], future_values_all['Month of the year'],future_values_all['Load forecast'], future_values_all['Generation forecast']))).reshape(1, -1, nb_time_feat)



#number_of_days =0
#for i in range(number_of_days):
#date = date + pd.Timedelta(hours=24)
#past_values_all_2 = train_dataset.filter(lambda x: date - pd.Timedelta(days = context) <= x['Date'].date() < date)
#past_observed_mask_temp = torch.ones(24*context)
#past_observed_mask=torch.cat((past_observed_mask, past_observed_mask_temp), dim=0)#.reshape(i+3, 24*context, nb_time_feat)
#past_time_features = torch.cat((past_time_features, torch.tensor(list(zip(past_values_all_2['Day of the week'], past_values_all_2['Hour of the day'],past_values_all_2['Day of the year'], past_values_all_2['Year'], past_values_all_2['Month of the year'], past_values_all_2['Load forecast'], past_values_all_2['Generation forecast'])))))#.reshape(i+3, 24*16, 2)
#past_values = torch.cat((past_values, torch.tensor(past_values_all_2['Price'])))#.reshape(i+3, 24*context, 3)

#future_values_all_2 = train_dataset.filter(lambda x: x['Date'].date() == date)
#future_values = torch.cat((future_values, torch.tensor(future_values_all_2['Price'])))#.reshape(i+3, 24, nb_time_feat)
#future_time_features = torch.cat((future_time_features, torch.tensor(list(zip(future_values_all_2['Day of the week'], future_values_all_2['Hour of the day'],future_values_all_2['Day of the year'], future_values_all_2['Year'], future_values_all_2['Month of the year'], future_values_all_2['Load forecast'], future_values_all_2['Generation forecast'])))))#.reshape(i+3, 24, 2)

#past_values = past_values.reshape(number_of_days+1, 24*context)
#past_time_features = past_time_features.reshape(number_of_days+1, 24*context, nb_time_feat)
#past_observed_mask = past_observed_mask.reshape(number_of_days+1, 24*context)

#future_values = future_values.reshape(number_of_days+1, 24)
#future_time_features = future_time_features.reshape(number_of_days+1, 24, nb_time_feat)
print(past_time_features.shape)
print(past_values.shape)
print(past_observed_mask.shape)

print(future_time_features.shape)
print(future_values.shape)

past_values = past_values.to(device)
past_time_features = past_time_features.to(device)
past_observed_mask = past_observed_mask.to(device)
future_values = future_values.to(device)
future_time_features = future_time_features.to(device)




# Initializing a Time Series Transformer configuration with 24 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=24, context_length=24*(context-1)+17, input_size=1, output_size=1, num_time_features=nb_time_feat )

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerForPrediction(configuration)

# Accessing the model configuration
configuration = model.config

model = model.to(device)



# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_values=future_values,
    future_time_features=future_time_features
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_time_features=future_time_features
)

mean_prediction = outputs.sequences.mean(dim=1).detach().cpu()
df_pred = pd.DataFrame(mean_prediction.numpy())

df_pred.to_csv('/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/forecasts/transformer_pred.csv')

Starting making prediction
   Unnamed: 0                Date  Load forecast  Generation forecast  Price  \
0           0 2017-01-01 00:00:00        73650.0              69539.0  58.23   
1           1 2017-01-01 01:00:00        72350.0              67376.0  51.95   
2           2 2017-01-01 02:00:00        68750.0              65412.0  47.27   
3           3 2017-01-01 03:00:00        65900.0              64557.0  45.49   
4           4 2017-01-01 04:00:00        65000.0              64552.0  44.50   

   Hour of the day  Day of the week  Day of the year  Year  Month of the year  
0                0                6                1  2017                  1  
1                1                6                1  2017                  1  
2                2                6                1  2017                  1  
3                3                6                1  2017                  1  
4                4                6                1  2017                  1  


Filter:   0%|          | 0/52584 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52584 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35064 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35064 [00:00<?, ? examples/s]

torch.Size([1, 4320, 7])
torch.Size([1, 4320])
torch.Size([1, 4320])
torch.Size([1, 24, 7])
torch.Size([1, 24])
